In [87]:
import pretty_midi as pm
from datasets import MidiToken
from utils import getNotes
import miditoolkit as mtk

# check pretty midi piano roll for instrument 1
def melodyRep(midi):
    '''
    Given a MIDI file, convert into a sequence of MIDI events. If note is in melody_notes have a NOTE_ON_MELODY/NOTE_OFF_MELODY token

        Parameters:
            midi_path (str/Path): Input MIDI filename
        
        Returns:
            arr (list): List of MidiToken event objects
    '''
    arr = []
    if not isinstance(midi, str):
        midi_data = midi
    else:
        midi_path = midi
        midi_data = pm.PrettyMIDI(midi_path)
        
    timestep=1000
    x = midi_data.get_piano_roll(fs=timestep) # shape=(pitch, length/timestep)
    melody_pr=midi_data.instruments[0].get_piano_roll(fs=timestep)

    # set non-zero velocities to 80
    x[x>0]=80
    melody_pr[melody_pr>0]=80


    # assert melody_pr.shape==x.shape
    
    active_notes = [] # unended NOTE_ON pitches
    time_acc = -10 # track time passed (ms) since last TIME_SHIFT (start at -10 to offset first increment)
    curr_vel = 0 # last SET_VELOCITY value 

    num_melody=0
    num_other=0
    # Iterate over timesteps
    for t in range(x.shape[1]):
        time_acc += 10
        for p in range(x.shape[0]):
            # When a note starts
            if x[p,t] and p not in active_notes:
                active_notes.append(p)
                if time_acc:
                    arr.append(MidiToken("TIME_SHIFT", time_acc))
                    time_acc = 0
                if (x[p,t]//4)*4 != curr_vel:
                    curr_vel = (x[p,t]//4)*4
                    arr.append(MidiToken("SET_VELOCITY", curr_vel))
                if t<melody_pr.shape[1] and melody_pr[p,t]:
                    arr.append(MidiToken("NOTE_ON_MELODY", p))
                    num_melody+=1
                else:
                    arr.append(MidiToken("NOTE_ON", p))
                    num_other+=1
            # When a note ends
            elif not x[p,t] and p in active_notes:
                if time_acc:
                    arr.append(MidiToken("TIME_SHIFT", time_acc))
                    time_acc = 0
                active_notes.remove(p)
                arr.append(MidiToken("NOTE_OFF", p))
        if time_acc == 1000:
            arr.append(MidiToken("TIME_SHIFT", 1000))
            time_acc = 0
    # Write final NOTE_OFFs and NOTE_OFF_MELODYs
    if active_notes:
        time_acc += 10
        arr.append(MidiToken("TIME_SHIFT", time_acc))
        for p in active_notes:
            if p != -1:
                arr.append(MidiToken("NOTE_OFF", p))
    print(f"Melody: {num_melody}, Other: {num_other}")
    return arr

In [88]:
midi_path="/Users/rohansharma/Desktop/Projects/Intern/score2performance/Sample_Data/asap-dataset/Bach/Fugue/bwv_848/midi_score.mid"

In [89]:
# read midi file and keep first 10 notes
midi_obj = mtk.midi.parser.MidiFile(midi_path)
melody_notes=getNotes(midi_obj, 0)
other_notes=getNotes(midi_obj, 1)

# keep first 5 melody notes and first 5 other notes
melody_notes=melody_notes[:5]
other_notes=other_notes[:10]

# create new midi object
new_midi_obj = mtk.midi.parser.MidiFile()
new_midi_obj.ticks_per_beat = midi_obj.ticks_per_beat
new_midi_obj.time_signature_changes = midi_obj.time_signature_changes

# create new instrument and add melody notes
melody_instr = mtk.midi.containers.Instrument(0, is_drum=False, name='melody')
melody_instr.notes = melody_notes
new_midi_obj.instruments.append(melody_instr)

# create new instrument and add other notes
other_instr = mtk.midi.containers.Instrument(1, is_drum=False, name='other')
other_instr.notes = other_notes
new_midi_obj.instruments.append(other_instr)

# save new midi file
new_midi_obj.dump("test.mid")

In [90]:
test_file="/Users/rohansharma/Desktop/Projects/Intern/score2performance/Store/Score2Melody/Bach/Fugue/bwv_846/Shi05M_score.mid"
out=melodyRep(test_file)
# retain note_on_melody and note_on
out=[x for x in out if x.type in ["NOTE_ON_MELODY", "NOTE_ON"]]

prettymidi_other_notes = [token.value for token in out if token.type == "NOTE_ON"]
prettymidi_melody_notes = [token.value for token in out if token.type == "NOTE_ON_MELODY"] 

Melody: 402, Other: 319


In [91]:
melody_notes=getNotes(test_file, 0)
other_notes=getNotes(test_file, 1)


mtk_melody_notes=[x.pitch for x in melody_notes]

mtk_other_notes=[x.pitch for x in other_notes]

print("Prettymidi melody notes: ", prettymidi_melody_notes)
print("Miditoolkit melody notes:", mtk_melody_notes)

print("Prettymidi other notes:  ", prettymidi_other_notes)
print("Miditoolkit other notes: ", mtk_other_notes)

Prettymidi melody notes:  [60, 62, 64, 65, 67, 65, 64, 69, 62, 67, 69, 67, 65, 64, 65, 67, 62, 69, 62, 71, 59, 72, 66, 74, 72, 67, 76, 69, 74, 64, 66, 76, 74, 72, 71, 55, 71, 57, 71, 59, 74, 60, 74, 76, 78, 60, 79, 64, 72, 69, 64, 74, 72, 71, 69, 67, 65, 48, 67, 50, 64, 52, 67, 53, 67, 69, 71, 53, 72, 67, 57, 64, 50, 57, 62, 60, 71, 57, 55, 53, 52, 53, 72, 50, 74, 50, 55, 47, 57, 59, 79, 77, 60, 81, 62, 60, 74, 57, 79, 55, 57, 81, 62, 77, 76, 64, 81, 60, 59, 83, 81, 79, 77, 59, 67, 62, 69, 59, 71, 55, 76, 59, 60, 74, 72, 79, 62, 76, 69, 57, 74, 78, 76, 74, 72, 79, 77, 43, 74, 45, 74, 62, 71, 69, 72, 71, 69, 48, 67, 52, 69, 67, 66, 72, 71, 69, 68, 52, 76, 48, 74, 71, 52, 68, 54, 68, 56, 71, 57, 66, 69, 59, 57, 56, 64, 60, 68, 54, 68, 59, 71, 76, 60, 59, 57, 71, 57, 71, 71, 72, 71, 72, 72, 56, 72, 69, 69, 60, 62, 55, 57, 59, 67, 65, 60, 69, 62, 60, 59, 43, 57, 69, 47, 65, 48, 67, 50, 48, 69, 71, 72, 66, 74, 72, 67, 52, 72, 48, 74, 76, 77, 71, 79, 77, 76, 57, 74, 72, 59, 61, 62, 81, 79, 7

In [92]:
# print lengths 
print("Prettymidi melody notes: ", len(prettymidi_melody_notes))
print("Miditoolkit melody notes:", len(mtk_melody_notes))

print("Prettymidi other notes:  ", len(prettymidi_other_notes))
print("Miditoolkit other notes: ", len(mtk_other_notes))

Prettymidi melody notes:  402
Miditoolkit melody notes: 417
Prettymidi other notes:   319
Miditoolkit other notes:  713
